# Import libraries

In [87]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np

from ipywidgets import interact

In [92]:
def fire_bounds(img, kernel='normal',kernel_opening=5, kernel_closing=5):
    global temp_masked
    global opening
    global closing
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    temp = np.copy(gray)
    temp[temp<0]=0
    temp[temp<0.4]=0
    temp_masked = cv2.multiply(gray, temp)
    
    # opening
    if kernel == 'normal':
        kernel_open = np.array([kernel_opening, kernel_opening])
        opening = cv2.morphologyEx(temp_masked, cv2.MORPH_OPEN, kernel_open)

        # closing
        kernel_close = np.array([kernel_closing, kernel_closing])
        closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel_close)
        masked_with_morpho = cv2.multiply(temp_masked, (closing//255))

        fig, ax = plt.subplots(1,3, figsize = (15,5))
        ax[0].imshow(np.squeeze(temp_masked),'gray')
        ax[0].set_title('masked img')
        ax[1].imshow(np.squeeze(opening), 'gray')
        ax[1].set_title('opening')
        ax[2].imshow(np.squeeze(closing), 'gray')
        ax[2].set_title('closing')

    else:
        kernel_open = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(kernel_opening, kernel_opening))
        opening = cv2.morphologyEx(temp_masked, cv2.MORPH_OPEN, kernel_open)

        # closing
        kernel_close = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(kernel_closing, kernel_closing))
        closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel_close)
        masked_with_morpho = cv2.multiply(temp_masked, (closing//255))
        
        fig, ax = plt.subplots(1,3, figsize = (15,5))
        ax[0].imshow(np.squeeze(temp_masked),'gray')
        ax[0].set_title('masked img')
        ax[1].imshow(np.squeeze(opening), 'gray')
        ax[1].set_title('opening')
        ax[2].imshow(np.squeeze(closing), 'gray')
        ax[2].set_title('closing')

        for axs in ax:
            axs.axis('Off')

In [93]:
DATA_DIR = '/home/victor/Mestrado/Disciplinas/Visão Computacional/Projeto Final/bowfire/BowFire'
IMG_DIR = os.path.join(DATA_DIR, 'images')
MASK_DIR = os.path.join(DATA_DIR, 'masks')

In [94]:
list_imgs = [os.path.join(IMG_DIR, x) for x in os.listdir(IMG_DIR)]
list_masks = [os.path.join(MASK_DIR, x) for x in os.listdir(MASK_DIR)]

list_imgs = sorted(list_imgs)
list_masks = sorted(list_masks)

print('Total de imagens', len(list_imgs))
print('Total de máscaras', len(list_masks))

Total de imagens 119
Total de máscaras 119


In [95]:
@interact(count = (0, len(list_imgs)), 
          kernel = ['normal','ellipse'],
          kernel_opening = (1,20,1), kernel_closing = (1,30,1))
def view_images(count, kernel, kernel_opening, kernel_closing):
    global img
    global mask
    fig, ax = plt.subplots(1,2, figsize=(10,5))
    img = plt.imread(list_imgs[count])
    mask = plt.imread(list_masks[count])
    
    ax[0].imshow(img)
    ax[0].set_title('original')
    ax[1].imshow(mask)
    ax[1].set_title('mask')
    
    fire_bounds(img, kernel_opening, kernel_closing)
    
    for axs in ax:
        axs.axis('Off')
        
# open = 3
# close = 20

interactive(children=(IntSlider(value=59, description='count', max=119), Dropdown(description='kernel', option…

In [97]:
pip install hdbscan


     |████████████████████████████████| 6.4 MB 1.6 MB/s eta 0:00:01     |██████████████████▉             | 3.8 MB 1.8 MB/s eta 0:00:02     |████████████████████████████▏   | 5.7 MB 1.6 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.27-cp38-cp38-linux_x86_64.whl size=3858126 sha256=98af0d4fcf97cf50ec15d3b20ae4ff434a27d2292b7190d57585826f19c2d425
  Stored in directory: /home/victor/.cache/pip/wheels/26/f2/c2/eab587fff76dc9ffc9a9bf3ca0e44e26d2ef6425264492df65
Successfully built hdbscan
Note: you may need to restart the kernel to use updated packages.


In [ ]:
clusterer = hdbscan.HDBSCAN(metric = 'mahalanobis')
clusterer.fit(img)
